The wand transformation is a very popular approach in insurance pricing. Although not used for the final results, it could be utilized for further research, if for example cat bonds are compared to reinsurance.
However, functions would have to be revise before that

In [ ]:
from scipy.stats import norm
import numpy as np
from scipy.interpolate import interp1d


losses = rel_simulated_ncf.copy()
wins = rel_simulated_ncf.copy()
losses[losses > 0] = 0
losses = losses * -1
wins = wins[wins > 0] 

def wang_transform(losses, lambda_value):
    """
    Apply Wang transformation to adjust probabilities for risk aversion.
    
    Parameters:
    - losses: List or array of relative losses (values between 0 and 1).
    - lambda_value: Risk aversion parameter (positive for risk-averse, negative for risk-seeking).
    
    Returns:
    - Transformed probabilities reflecting risk aversion.
    """

    # Step 1: Compute the empirical CDF of the returns
    sorted_losses = np.sort(losses)
    n = len(sorted_losses)
    exceedance_prob = 1 - (np.arange(1, n + 1) / n)
    
    # Step 2: Apply the Wang transformation to the probabilities
    wang_exceedance_prob = norm.cdf(norm.ppf(exceedance_prob) + lambda_value)
    
    # Step 3: Interpolate the transformed probabilities to find transformed returns
    #transformed_losses = np.interp(wang_exceedance_prob, exceedance_prob, sorted_losses)

    interp_function = interp1d(exceedance_prob, sorted_losses, kind='linear', fill_value='extrapolate')
    transformed_losses = interp_function(wang_exceedance_prob)

    return transformed_losses, wang_exceedance_prob, exceedance_prob, sorted_losses

# Example usage:
lambda_value = 0.5  # Risk aversion parameter
transformed_losses, wang_exceedance_prob, exceedance_prob, sorted_losses = wang_transform(rel_simulated_ncf, lambda_value)


In [ ]:
plt.plot(exceedance_prob, color='green')
plt.plot(wang_exceedance_prob, color='pink')
plt.plot(sorted_losses, color='orange')
plt.plot(transformed_losses, color='red')
plt.show

In [ ]:
td_ncf = np.std(transformed_losses)
exp_ret = transformed_losses.mean()
sharpe_ratio = (exp_ret - rf) / td_ncf
print('The expected sharp ratio is: ', sharpe_ratio)